In [43]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [44]:
import os
import json
import time
from datetime import datetime
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

from lightgbm import LGBMRegressor

# ============================================================
# K-League Track1 - LGBM Direct Regression (Wide Feature + CV)
# ------------------------------------------------------------
# ✅ 목표: end_x, end_y를 BIN 없이 "직접 회귀"로 예측
# ✅ 핵심: 마지막 K 이벤트를 wide feature로 펼친 뒤,
#         LGBMRegressor로 target_x, target_y 각각 학습
# ✅ 옵션: KFold로 CV(OOF) 점수 확인 후, 전체 학습 → Test 예측
#
# ⚠️ BASE_PATH 및 파일 경로는 반드시 그대로 유지
# ============================================================

# ----------------------
# 0. 설정 (경로/하이퍼파라미터)
# ----------------------
BASE_PATH = "open_track1/"
PATH_TRAIN = os.path.join(BASE_PATH, "train.csv")
PATH_TEST = os.path.join(BASE_PATH, "test.csv")
PATH_MATCH_INFO = os.path.join(BASE_PATH, "match_info.csv")
PATH_SAMPLE_SUB = os.path.join(BASE_PATH, "sample_submission.csv")

K = 30   # 마지막 K 이벤트 사용 (20~32 사이 선택)

# CV 설정
USE_CV = True
N_SPLITS = 5
SEED = 42

# LGBM 기본 파라미터 (너무 과튜닝하지 말고, 안정적으로 시작)
LGB_PARAMS_BASE = dict(
    n_estimators=5000,          # early stopping으로 실제 사용 개수 자동 결정
    learning_rate=0.03,
    num_leaves=64,
    max_depth=-1,
    min_child_samples=50,       # (=min_data_in_leaf 비슷)
    subsample=0.8,              # bagging_fraction
    subsample_freq=1,           # bagging_freq
    colsample_bytree=0.8,       # feature_fraction
    reg_alpha=0.0,
    reg_lambda=0.0,
    n_jobs=-1
)

# OOF 앙상블을 위한 하이퍼파라미터 조합
# seed: [42, 2023, 777]
# num_leaves: [48, 64, 96]
# min_child_samples: [30, 50]
ENSEMBLE_SEEDS = [42, 2023, 777]
ENSEMBLE_NUM_LEAVES = [48, 64, 96]
ENSEMBLE_MIN_CHILD_SAMPLES = [30, 50]

# 앙상블 사용 여부
USE_ENSEMBLE = False  # 피처 실험 시에는 False, 최종 제출 시 True로 변경

# ============================================================
# 자동화된 피처 실험 파이프라인
# ============================================================
# 모든 피처를 순서대로 자동으로 테스트
RUN_FEATURE_EXPERIMENTS = True  # True: 자동 실험, False: 수동 플래그 제어

# ============================================================
# 피처 고도화 실험 플래그 (One-by-One 검증)
# ============================================================
# ⚠️ RUN_FEATURE_EXPERIMENTS = True일 때는 자동으로 제어됨
# ⚠️ RUN_FEATURE_EXPERIMENTS = False일 때만 수동으로 설정

# 1️⃣ Recency 가중 피처 (최우선, ★★★★★)
USE_RECENCY_WEIGHT = False
RECENCY_ALPHA = 0.1  # exp(-alpha * (K - pos_in_K))

# 2️⃣ 최근 이동 방향 요약 피처 (★★★★☆)
USE_DIRECTION_SUMMARY = False

# 3️⃣ 필드 경계 인식 강화 피처 (★★★★☆)
USE_FIELD_BOUNDARY = False
FIELD_BOUNDARY_THRESHOLD = 5.0  # T = 5m

# 4️⃣ 최근 속도 기반 이동 한계 피처 (★★★☆☆)
USE_SPEED_CONSTRAINT = False

# 5️⃣ 코너/사이드 상황 강조 피처 (★★★☆☆)
USE_CORNER_CONTEXT = False

# 실험할 피처 목록 (순서대로)
FEATURE_EXPERIMENTS = [
    {"name": "baseline", "USE_RECENCY_WEIGHT": False, "USE_DIRECTION_SUMMARY": False, 
     "USE_FIELD_BOUNDARY": False, "USE_SPEED_CONSTRAINT": False, "USE_CORNER_CONTEXT": False},
    {"name": "Recency", "USE_RECENCY_WEIGHT": True, "USE_DIRECTION_SUMMARY": False, 
     "USE_FIELD_BOUNDARY": False, "USE_SPEED_CONSTRAINT": False, "USE_CORNER_CONTEXT": False},
    {"name": "Direction", "USE_RECENCY_WEIGHT": False, "USE_DIRECTION_SUMMARY": True, 
     "USE_FIELD_BOUNDARY": False, "USE_SPEED_CONSTRAINT": False, "USE_CORNER_CONTEXT": False},
    {"name": "FieldBoundary", "USE_RECENCY_WEIGHT": False, "USE_DIRECTION_SUMMARY": False, 
     "USE_FIELD_BOUNDARY": True, "USE_SPEED_CONSTRAINT": False, "USE_CORNER_CONTEXT": False},
    {"name": "SpeedConstraint", "USE_RECENCY_WEIGHT": False, "USE_DIRECTION_SUMMARY": False, 
     "USE_FIELD_BOUNDARY": False, "USE_SPEED_CONSTRAINT": True, "USE_CORNER_CONTEXT": False},
    {"name": "CornerContext", "USE_RECENCY_WEIGHT": False, "USE_DIRECTION_SUMMARY": False, 
     "USE_FIELD_BOUNDARY": False, "USE_SPEED_CONSTRAINT": False, "USE_CORNER_CONTEXT": True},
]

# ----------------------
# 1. 데이터 로드
# ----------------------
train = pd.read_csv(PATH_TRAIN)
test_index = pd.read_csv(PATH_TEST)
match_info = pd.read_csv(PATH_MATCH_INFO)  # 현재 코드에서는 직접 사용 안 함 (원본 유지)
sample_sub = pd.read_csv(PATH_SAMPLE_SUB)

# test는 episode별 csv로 따로 존재 → 합쳐서 events로 만들기
test_events_list = []
for _, row in test_index.iterrows():
    test_path = os.path.join(BASE_PATH, row["path"].lstrip("./"))
    df_ep = pd.read_csv(test_path)
    test_events_list.append(df_ep)

test_events = pd.concat(test_events_list, ignore_index=True)

train["is_train"] = 1
test_events["is_train"] = 0

events = pd.concat([train, test_events], ignore_index=True)

# ----------------------
# 2. 기본 정렬 + episode 내 인덱스
# ----------------------
events = events.sort_values(["game_episode", "time_seconds", "action_id"]).reset_index(drop=True)

events["event_idx"] = events.groupby("game_episode").cumcount()
events["n_events"] = events.groupby("game_episode")["event_idx"].transform("max") + 1
events["ep_idx_norm"] = events["event_idx"] / (events["n_events"] - 1).clip(lower=1)

# ----------------------
# 3. 시간/공간 feature
# ----------------------
events["prev_time"] = events.groupby("game_episode")["time_seconds"].shift(1)
events["dt"] = (events["time_seconds"] - events["prev_time"]).fillna(0.0)

events["dx"] = events["end_x"] - events["start_x"]
events["dy"] = events["end_y"] - events["start_y"]
events["dist"] = np.sqrt(events["dx"]**2 + events["dy"]**2)

# dt=0 보호
events["speed"] = events["dist"] / events["dt"].replace(0, 1e-3)

# zone / lane
events["x_zone"] = (events["start_x"] / (105/7)).astype(int).clip(0, 6)
events["lane"] = pd.cut(
    events["start_y"],
    bins=[0, 68/3, 2*68/3, 68],
    labels=[0, 1, 2],
    include_lowest=True
).astype(int)

# ----------------------
# 3-1. 코너 관련 피처 (Corner Features)
# ----------------------
events["angle_to_goal"] = np.arctan2(
    34 - events["start_y"],
    105 - events["start_x"]
) * 180 / np.pi

events["dist_corner_top"] = np.sqrt((105 - events["start_x"])**2 + (0 - events["start_y"])**2)
events["dist_corner_bottom"] = np.sqrt((105 - events["start_x"])**2 + (68 - events["start_y"])**2)
events["dist_to_nearest_corner"] = events[["dist_corner_top", "dist_corner_bottom"]].min(axis=1)

events["is_corner_area"] = ((events["start_x"] > 100) &
                            ((events["start_y"] < 5) | (events["start_y"] > 63))).astype(int)

events["angle_goal_x_corner"] = events["angle_to_goal"] * events["is_corner_area"]
events["dist_corner_x_angle"] = events["dist_to_nearest_corner"] * events["angle_to_goal"]

# ----------------------
# 3-2. 새로운 핵심 피처 3개 (고득점용)
# ----------------------
events["dist_to_sideline"] = events["start_y"].apply(lambda y: min(y, 68-y))

# 라디안 단위
events["angle_to_goal_center"] = np.arctan2(
    34 - events["start_y"],
    105 - events["start_x"]
)

events["time_pos_inter"] = events["ep_idx_norm"] * events["start_x"]

print("✓ 핵심 피처 3개 추가 완료 (dist_to_sideline, angle_to_goal_center, time_pos_inter)")

# ----------------------
# 4. 라벨 및 episode-level 메타 (train 전용)
# ----------------------
train_events = events[events["is_train"] == 1].copy()

last_events = (
    train_events
    .groupby("game_episode", as_index=False)
    .tail(1)
    .copy()
)

labels = last_events[["game_episode", "end_x", "end_y"]].rename(
    columns={"end_x": "target_x", "end_y": "target_y"}
)

ep_meta = last_events[["game_episode", "game_id", "team_id", "is_home", "period_id", "time_seconds"]].copy()
ep_meta = ep_meta.rename(columns={"team_id": "final_team_id"})

ep_meta["game_clock_min"] = np.where(
    ep_meta["period_id"] == 1,
    ep_meta["time_seconds"] / 60.0,
    45.0 + ep_meta["time_seconds"] / 60.0
)

# ----------------------
# 5. 공격 팀 플래그 (final_team vs 상대)
# ----------------------
events = events.merge(
    ep_meta[["game_episode", "final_team_id"]],
    on="game_episode",
    how="left"
)
events["is_final_team"] = (events["team_id"] == events["final_team_id"]).astype(int)

# ----------------------
# 6. 입력용 events에서 마지막 이벤트 타깃 정보 가리기
# ----------------------
events["last_idx"] = events.groupby("game_episode")["event_idx"].transform("max")
events["is_last"] = (events["event_idx"] == events["last_idx"]).astype(int)

mask_last = events["is_last"] == 1
for col in ["end_x", "end_y", "dx", "dy", "dist", "speed"]:
    events.loc[mask_last, col] = np.nan

# ----------------------
# 7. 카테고리 인코딩 (type_name, result_name, team_id 등)
# ----------------------
events["type_name"] = events["type_name"].fillna("__NA_TYPE__")
events["result_name"] = events["result_name"].fillna("__NA_RES__")

le_type = LabelEncoder()
le_res = LabelEncoder()

events["type_id"] = le_type.fit_transform(events["type_name"])
events["res_id"] = le_res.fit_transform(events["result_name"])

if events["team_id"].dtype == "object":
    le_team = LabelEncoder()
    events["team_id_enc"] = le_team.fit_transform(events["team_id"])
else:
    events["team_id_enc"] = events["team_id"].astype(int)

# ----------------------
# 8. 마지막 K 이벤트만 사용 (lastK)
# ----------------------
events["rev_idx"] = events.groupby("game_episode")["event_idx"].transform(
    lambda s: s.max() - s
)
lastK = events[events["rev_idx"] < K].copy()

def assign_pos_in_K(df):
    df = df.sort_values("event_idx")  # 오래된 → 최근
    L = len(df)
    df = df.copy()
    df["pos_in_K"] = np.arange(K - L, K)
    return df

lastK = lastK.groupby("game_episode", group_keys=False).apply(assign_pos_in_K)

# ----------------------
# 9. wide feature pivot
# ----------------------
num_cols = [
    "start_x", "start_y",
    "end_x", "end_y",
    "dx", "dy", "dist", "speed",
    "dt",
    "ep_idx_norm",
    "x_zone", "lane",
    "is_final_team",
    # 코너 관련
    "angle_to_goal",
    "dist_to_nearest_corner",
    "is_corner_area",
    "angle_goal_x_corner",
    "dist_corner_x_angle",
    # 핵심 3개
    "dist_to_sideline",
    "angle_to_goal_center",
    "time_pos_inter",
]

cat_cols = [
    "type_id",
    "res_id",
    "team_id_enc",
    "is_home",
    "period_id",
    "is_last",
]

feature_cols = num_cols + cat_cols
wide = lastK[["game_episode", "pos_in_K"] + feature_cols].copy()

wide_num = wide.pivot_table(index="game_episode", columns="pos_in_K", values=num_cols, aggfunc="first")
wide_cat = wide.pivot_table(index="game_episode", columns="pos_in_K", values=cat_cols, aggfunc="first")

wide_num.columns = [f"{c}_{int(pos)}" for (c, pos) in wide_num.columns]
wide_cat.columns = [f"{c}_{int(pos)}" for (c, pos) in wide_cat.columns]

X = pd.concat([wide_num, wide_cat], axis=1).reset_index()

# episode-level 메타 붙이기
X = X.merge(
    ep_meta[["game_episode", "game_id", "game_clock_min", "final_team_id", "is_home", "period_id"]],
    on="game_episode",
    how="left"
)

# train 라벨 붙이기
X = X.merge(labels, on="game_episode", how="left")

# ----------------------
# 10. train/test 분리
# ----------------------
train_mask = X["game_episode"].isin(labels["game_episode"])
X_train = X[train_mask].copy()
X_test = X[~train_mask].copy()

y_train_x = X_train["target_x"].astype(float).values
y_train_y = X_train["target_y"].astype(float).values

drop_cols = ["game_episode", "game_id", "target_x", "target_y"]

X_train_feat = X_train.drop(columns=drop_cols)
X_test_feat = X_test.drop(columns=[c for c in drop_cols if c in X_test.columns])

# 결측 처리
X_train_feat = X_train_feat.fillna(0)
X_test_feat = X_test_feat.fillna(0)

# is_home 컬럼을 숫자로 변환 (LightGBM 호환성)
for col in X_train_feat.columns:
    if "is_home" in col and X_train_feat[col].dtype == "object":
        X_train_feat[col] = pd.to_numeric(X_train_feat[col], errors="coerce").fillna(0).astype(int)
        if col in X_test_feat.columns:
            X_test_feat[col] = pd.to_numeric(X_test_feat[col], errors="coerce").fillna(0).astype(int)
    
    # period_id 컬럼도 확인
    if "period_id" in col and X_train_feat[col].dtype == "object":
        X_train_feat[col] = pd.to_numeric(X_train_feat[col], errors="coerce").fillna(1).astype(int)
        if col in X_test_feat.columns:
            X_test_feat[col] = pd.to_numeric(X_test_feat[col], errors="coerce").fillna(1).astype(int)

# ----------------------
# 9-1. 피처 고도화 (One-by-One 검증)
# ----------------------
def add_advanced_features(X_train_feat, X_test_feat, X_train, X_test, events_df, K_val):
    """
    피처 고도화 실험용 피처 추가
    각 플래그에 따라 하나씩만 추가
    """
    X_train_feat = X_train_feat.copy()
    X_test_feat = X_test_feat.copy()
    
    # 1️⃣ Recency 가중 피처
    if USE_RECENCY_WEIGHT:
        print("  ✓ Recency 가중 피처 추가 중...")
        # pd.concat을 사용하여 한 번에 병합 (성능 최적화)
        weighted_train_cols = {}
        weighted_test_cols = {}
        
        # 마지막 이벤트의 pos_in_K = K-1
        for pos in range(K_val):
            recency_weight = np.exp(-RECENCY_ALPHA * (K_val - pos))
            
            # dx, dy, speed, dist에 가중치 적용
            for feat_base in ["dx", "dy", "speed", "dist"]:
                col_name = f"{feat_base}_{pos}"
                if col_name in X_train_feat.columns:
                    weighted_train_cols[f"{feat_base}_weighted_{pos}"] = X_train_feat[col_name] * recency_weight
                if col_name in X_test_feat.columns:
                    weighted_test_cols[f"{feat_base}_weighted_{pos}"] = X_test_feat[col_name] * recency_weight
        
        # 한 번에 병합
        if weighted_train_cols:
            weighted_train_df = pd.DataFrame(weighted_train_cols, index=X_train_feat.index)
            X_train_feat = pd.concat([X_train_feat, weighted_train_df], axis=1)
        if weighted_test_cols:
            weighted_test_df = pd.DataFrame(weighted_test_cols, index=X_test_feat.index)
            X_test_feat = pd.concat([X_test_feat, weighted_test_df], axis=1)
    
    # 2️⃣ 최근 이동 방향 요약 피처
    if USE_DIRECTION_SUMMARY:
        print("  ✓ 최근 이동 방향 요약 피처 추가 중...")
        # 마지막 3개, 5개 이벤트의 평균
        for last_n in [3, 5]:
            dx_cols = [f"dx_{K_val-1-i}" for i in range(last_n) if f"dx_{K_val-1-i}" in X_train_feat.columns]
            dy_cols = [f"dy_{K_val-1-i}" for i in range(last_n) if f"dy_{K_val-1-i}" in X_train_feat.columns]
            
            if dx_cols:
                X_train_feat[f"mean_dx_last{last_n}"] = X_train_feat[dx_cols].mean(axis=1)
                X_test_feat[f"mean_dx_last{last_n}"] = X_test_feat[dx_cols].mean(axis=1) if all(c in X_test_feat.columns for c in dx_cols) else 0
            
            if dy_cols:
                X_train_feat[f"mean_dy_last{last_n}"] = X_train_feat[dy_cols].mean(axis=1)
                X_test_feat[f"mean_dy_last{last_n}"] = X_test_feat[dy_cols].mean(axis=1) if all(c in X_test_feat.columns for c in dy_cols) else 0
        
        # angle_mean_last5 (angle_to_goal 평균)
        angle_cols = [f"angle_to_goal_{K_val-1-i}" for i in range(5) if f"angle_to_goal_{K_val-1-i}" in X_train_feat.columns]
        if angle_cols:
            X_train_feat["angle_mean_last5"] = X_train_feat[angle_cols].mean(axis=1)
            X_test_feat["angle_mean_last5"] = X_test_feat[angle_cols].mean(axis=1) if all(c in X_test_feat.columns for c in angle_cols) else 0
    
    # 3️⃣ 필드 경계 인식 강화 피처
    if USE_FIELD_BOUNDARY:
        print("  ✓ 필드 경계 인식 강화 피처 추가 중...")
        # 마지막 이벤트의 start_x, start_y 사용
        last_start_x_col = f"start_x_{K_val-1}"
        last_start_y_col = f"start_y_{K_val-1}"
        
        if last_start_x_col in X_train_feat.columns:
            X_train_feat["dist_to_goal_line"] = 105 - X_train_feat[last_start_x_col]
            X_test_feat["dist_to_goal_line"] = 105 - X_test_feat[last_start_x_col] if last_start_x_col in X_test_feat.columns else 0
            
            X_train_feat["is_near_goal_line"] = (X_train_feat["dist_to_goal_line"] < FIELD_BOUNDARY_THRESHOLD).astype(int)
            X_test_feat["is_near_goal_line"] = (X_test_feat["dist_to_goal_line"] < FIELD_BOUNDARY_THRESHOLD).astype(int)
        
        # dist_to_sideline은 이미 있을 수 있음
        last_dist_sideline_col = f"dist_to_sideline_{K_val-1}"
        if last_dist_sideline_col in X_train_feat.columns:
            X_train_feat["is_near_sideline"] = (X_train_feat[last_dist_sideline_col] < FIELD_BOUNDARY_THRESHOLD).astype(int)
            X_test_feat["is_near_sideline"] = (X_test_feat[last_dist_sideline_col] < FIELD_BOUNDARY_THRESHOLD).astype(int) if last_dist_sideline_col in X_test_feat.columns else 0
    
    # 4️⃣ 최근 속도 기반 이동 한계 피처
    if USE_SPEED_CONSTRAINT:
        print("  ✓ 최근 속도 기반 이동 한계 피처 추가 중...")
        # 마지막 이벤트의 speed, dt
        last_speed_col = f"speed_{K_val-1}"
        last_dt_col = f"dt_{K_val-1}"
        
        if last_speed_col in X_train_feat.columns and last_dt_col in X_train_feat.columns:
            X_train_feat["last_speed"] = X_train_feat[last_speed_col]
            X_train_feat["last_dt"] = X_train_feat[last_dt_col]
            X_train_feat["max_possible_move"] = X_train_feat["last_speed"] * X_train_feat["last_dt"]
            
            if last_speed_col in X_test_feat.columns and last_dt_col in X_test_feat.columns:
                X_test_feat["last_speed"] = X_test_feat[last_speed_col]
                X_test_feat["last_dt"] = X_test_feat[last_dt_col]
                X_test_feat["max_possible_move"] = X_test_feat["last_speed"] * X_test_feat["last_dt"]
    
    # 5️⃣ 코너/사이드 상황 강조 피처
    if USE_CORNER_CONTEXT:
        print("  ✓ 코너/사이드 상황 강조 피처 추가 중...")
        # 코너 영역 판단 (x > 100 and (y < 10 or y > 58))
        last_start_x_col = f"start_x_{K_val-1}"
        last_start_y_col = f"start_y_{K_val-1}"
        
        if last_start_x_col in X_train_feat.columns and last_start_y_col in X_train_feat.columns:
            X_train_feat["is_corner_area"] = (
                (X_train_feat[last_start_x_col] > 100) & 
                ((X_train_feat[last_start_y_col] < 10) | (X_train_feat[last_start_y_col] > 58))
            ).astype(int)
            
            if last_start_x_col in X_test_feat.columns and last_start_y_col in X_test_feat.columns:
                X_test_feat["is_corner_area"] = (
                    (X_test_feat[last_start_x_col] > 100) & 
                    ((X_test_feat[last_start_y_col] < 10) | (X_test_feat[last_start_y_col] > 58))
                ).astype(int)
            
            # 상호작용 피처
            last_angle_col = f"angle_to_goal_{K_val-1}"
            last_dist_sideline_col = f"dist_to_sideline_{K_val-1}"
            
            if last_angle_col in X_train_feat.columns:
                X_train_feat["angle_to_goal_x_corner"] = X_train_feat[last_angle_col] * X_train_feat["is_corner_area"]
                if last_angle_col in X_test_feat.columns:
                    X_test_feat["angle_to_goal_x_corner"] = X_test_feat[last_angle_col] * X_test_feat["is_corner_area"]
            
            if last_dist_sideline_col in X_train_feat.columns:
                X_train_feat["dist_to_sideline_x_corner"] = X_train_feat[last_dist_sideline_col] * X_train_feat["is_corner_area"]
                if last_dist_sideline_col in X_test_feat.columns:
                    X_test_feat["dist_to_sideline_x_corner"] = X_test_feat[last_dist_sideline_col] * X_test_feat["is_corner_area"]
    
    return X_train_feat, X_test_feat

# ----------------------
# 11. 자동화된 피처 실험 파이프라인
# ----------------------
def euclidean_mean(y_true_x, y_true_y, y_pred_x, y_pred_y):
    dx = y_true_x - y_pred_x
    dy = y_true_y - y_pred_y
    return np.mean(np.sqrt(dx*dx + dy*dy))

def run_single_experiment(exp_config, X_train_feat_base, X_test_feat_base, X_train, X_test, 
                         y_train_x, y_train_y, events, K_val):
    """
    단일 피처 실험 실행 (상세 진행 상황 표시 포함)
    """
    exp_start_time = time.time()
    
    # 플래그 설정
    global USE_RECENCY_WEIGHT, USE_DIRECTION_SUMMARY, USE_FIELD_BOUNDARY
    global USE_SPEED_CONSTRAINT, USE_CORNER_CONTEXT
    
    USE_RECENCY_WEIGHT = exp_config["USE_RECENCY_WEIGHT"]
    USE_DIRECTION_SUMMARY = exp_config["USE_DIRECTION_SUMMARY"]
    USE_FIELD_BOUNDARY = exp_config["USE_FIELD_BOUNDARY"]
    USE_SPEED_CONSTRAINT = exp_config["USE_SPEED_CONSTRAINT"]
    USE_CORNER_CONTEXT = exp_config["USE_CORNER_CONTEXT"]
    
    # 피처 추가
    print("  📦 피처 생성 중...")
    feat_start = time.time()
    X_train_feat = X_train_feat_base.copy()
    X_test_feat = X_test_feat_base.copy()
    
    active_features = []
    if USE_RECENCY_WEIGHT:
        active_features.append("Recency")
    if USE_DIRECTION_SUMMARY:
        active_features.append("Direction")
    if USE_FIELD_BOUNDARY:
        active_features.append("FieldBoundary")
    if USE_SPEED_CONSTRAINT:
        active_features.append("SpeedConstraint")
    if USE_CORNER_CONTEXT:
        active_features.append("CornerContext")
    
    if active_features:
        X_train_feat, X_test_feat = add_advanced_features(X_train_feat, X_test_feat, X_train, X_test, events, K_val)
    feat_time = time.time() - feat_start
    print(f"  ✓ 피처 생성 완료 ({feat_time:.1f}초) | Train: {X_train_feat.shape[1]} features, Test: {X_test_feat.shape[1]} features")
    
    # CV 실행 (단일 모델)
    print(f"  🔄 CV 학습 시작 (n_splits={N_SPLITS})...")
    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    oof_pred_x = np.zeros(len(X_train_feat), dtype=float)
    oof_pred_y = np.zeros(len(X_train_feat), dtype=float)
    fold_scores = []
    
    LGB_PARAMS = LGB_PARAMS_BASE.copy()
    LGB_PARAMS["random_state"] = SEED
    
    for fold, (tr_idx, va_idx) in enumerate(kf.split(X_train_feat), 1):
        fold_start = time.time()
        print(f"    └─ Fold {fold}/{N_SPLITS} 학습 중...", end=" ", flush=True)
        
        X_tr, X_va = X_train_feat.iloc[tr_idx], X_train_feat.iloc[va_idx]
        y_tr_x, y_va_x = y_train_x[tr_idx], y_train_x[va_idx]
        y_tr_y, y_va_y = y_train_y[tr_idx], y_train_y[va_idx]
        
        model_x = LGBMRegressor(**LGB_PARAMS)
        model_x.fit(X_tr, y_tr_x, eval_set=[(X_va, y_va_x)], eval_metric="rmse", callbacks=[])
        
        model_y = LGBMRegressor(**LGB_PARAMS)
        model_y.fit(X_tr, y_tr_y, eval_set=[(X_va, y_va_y)], eval_metric="rmse", callbacks=[])
        
        pred_va_x = np.clip(model_x.predict(X_va), 0, 105)
        pred_va_y = np.clip(model_y.predict(X_va), 0, 68)
        
        oof_pred_x[va_idx] = pred_va_x
        oof_pred_y[va_idx] = pred_va_y
        
        fold_euc = euclidean_mean(y_va_x, y_va_y, pred_va_x, pred_va_y)
        fold_scores.append(fold_euc)
        fold_time = time.time() - fold_start
        print(f"mean_dist = {fold_euc:.6f} ({fold_time:.1f}초)")
    
    cv_mean = float(np.mean(fold_scores))
    cv_std = float(np.std(fold_scores))
    
    # 상세 지표 계산
    dist_all = np.sqrt((oof_pred_x - y_train_x)**2 + (oof_pred_y - y_train_y)**2)
    p90 = np.percentile(dist_all, 90)
    p95 = np.percentile(dist_all, 95)
    pred_std_x = np.std(oof_pred_x)
    pred_std_y = np.std(oof_pred_y)
    
    exp_time = time.time() - exp_start_time
    
    result = {
        "name": exp_config["name"],
        "features": ", ".join(active_features) if active_features else "없음",
        "cv_mean": cv_mean,
        "cv_std": cv_std,
        "p90": p90,
        "p95": p95,
        "pred_std_x": pred_std_x,
        "pred_std_y": pred_std_y,
        "timestamp": datetime.now().isoformat(),
        "elapsed_time": exp_time
    }
    
    print(f"  ✓ 실험 완료 (총 {exp_time:.1f}초)")
    return result

# 피처 추가 실행 (기본 피처만, 실험 전 준비)
X_train_feat_base = X_train_feat.copy()
X_test_feat_base = X_test_feat.copy()

# 자동화된 실험 실행
if RUN_FEATURE_EXPERIMENTS:
    # 실험 결과 저장 파일 경로
    EXPERIMENT_RESULTS_FILE = "feature_experiment_results.json"
    
    print("\n" + "=" * 80)
    print("🚀 자동화된 피처 실험 파이프라인 시작")
    print("=" * 80)
    print(f"총 {len(FEATURE_EXPERIMENTS)}개 실험 실행 예정")
    
    # 이전 결과 로드
    saved_results = {}
    if os.path.exists(EXPERIMENT_RESULTS_FILE):
        try:
            with open(EXPERIMENT_RESULTS_FILE, 'r', encoding='utf-8') as f:
                saved_data = json.load(f)
                saved_results = {exp['name']: exp for exp in saved_data.get('results', [])}
            print(f"📂 이전 실험 결과 로드 완료: {len(saved_results)}개 완료된 실험 발견")
        except Exception as e:
            print(f"⚠️  이전 결과 로드 실패: {e}")
            saved_results = {}
    
    experiment_results = []
    pipeline_start_time = time.time()
    
    for exp_idx, exp_config in enumerate(FEATURE_EXPERIMENTS, 1):
        exp_name = exp_config['name']
        
        # 이미 완료된 실험인지 확인
        if exp_name in saved_results:
            print(f"\n{'='*80}")
            print(f"⏭️  실험 {exp_idx}/{len(FEATURE_EXPERIMENTS)}: {exp_name} (이미 완료 - 스킵)")
            print(f"{'='*80}")
            print(f"  📊 이전 결과: mean_dist = {saved_results[exp_name]['cv_mean']:.6f} ± {saved_results[exp_name]['cv_std']:.6f}")
            experiment_results.append(saved_results[exp_name])
            continue
        
        print(f"\n{'='*80}")
        print(f"📌 실험 {exp_idx}/{len(FEATURE_EXPERIMENTS)}: {exp_name}")
        print(f"{'='*80}")
        
        if exp_name == 'baseline':
            print("📊 Baseline 모드 (추가 피처 없음)")
        else:
            print(f"🔧 활성화된 피처: {exp_name}")
        
        # 개별 실험 오류 처리
        try:
            result = run_single_experiment(
                exp_config, X_train_feat_base, X_test_feat_base, 
                X_train, X_test, y_train_x, y_train_y, events, K
            )
            experiment_results.append(result)
            
            # 결과 즉시 저장 (중간 저장)
            with open(EXPERIMENT_RESULTS_FILE, 'w', encoding='utf-8') as f:
                json.dump({
                    'results': experiment_results,
                    'last_updated': datetime.now().isoformat()
                }, f, indent=2, ensure_ascii=False)
            
            print(f"  💾 결과 저장 완료: {EXPERIMENT_RESULTS_FILE}")
            print(f"✅ {exp_name} 완료 | mean_dist = {result['cv_mean']:.6f} ± {result['cv_std']:.6f}")
            print(f"   p90 = {result['p90']:.6f}, p95 = {result['p95']:.6f}")
            
            # 예상 남은 시간 계산
            if exp_idx < len(FEATURE_EXPERIMENTS):
                elapsed = time.time() - pipeline_start_time
                avg_time_per_exp = elapsed / exp_idx
                remaining_exps = len(FEATURE_EXPERIMENTS) - exp_idx
                estimated_remaining = avg_time_per_exp * remaining_exps
                print(f"  ⏱️  예상 남은 시간: 약 {estimated_remaining/60:.1f}분 ({remaining_exps}개 실험 남음)")
        
        except KeyboardInterrupt:
            # 사용자가 의도적으로 중단한 경우
            print(f"\n\n{'='*80}")
            print("⚠️  사용자에 의해 실험 중단됨 (KeyboardInterrupt)")
            print("=" * 80)
            
            # 완료된 실험 결과 저장
            if experiment_results:
                with open(EXPERIMENT_RESULTS_FILE, 'w', encoding='utf-8') as f:
                    json.dump({
                        'results': experiment_results,
                        'last_updated': datetime.now().isoformat(),
                        'interrupted': True,
                        'interrupted_at': exp_name
                    }, f, indent=2, ensure_ascii=False)
                print(f"💾 완료된 {len(experiment_results)}개 실험 결과 저장 완료: {EXPERIMENT_RESULTS_FILE}")
                print(f"📌 중단된 실험: {exp_name}")
                print(f"💡 다음 실행 시 완료된 실험은 스킵되고 중단된 실험부터 재개됩니다.")
            else:
                print("⚠️  완료된 실험이 없어 저장할 결과가 없습니다.")
            
            print("=" * 80)
            raise  # KeyboardInterrupt를 다시 발생시켜 정상 종료
        
        except Exception as e:
            print(f"  ❌ 실험 실패: {exp_name}")
            print(f"  오류 내용: {str(e)}")
            print(f"  ⚠️  이 실험은 건너뛰고 다음 실험으로 진행합니다.")
            import traceback
            traceback.print_exc()
            
            # 완료된 실험 결과는 저장 (오류가 발생한 실험 제외)
            if experiment_results:
                with open(EXPERIMENT_RESULTS_FILE, 'w', encoding='utf-8') as f:
                    json.dump({
                        'results': experiment_results,
                        'last_updated': datetime.now().isoformat(),
                        'failed_experiment': exp_name
                    }, f, indent=2, ensure_ascii=False)
            continue
    
    # 전체 파이프라인 시간
    total_time = time.time() - pipeline_start_time
    print(f"\n{'='*80}")
    print(f"⏱️  전체 파이프라인 완료: {total_time/60:.1f}분 ({total_time:.1f}초)")
    print("=" * 80)
    
    # 결과 비교 테이블 출력
    print("\n" + "=" * 80)
    print("📊 피처 실험 결과 비교")
    print("=" * 80)
    print(f"{'실험명':<20} {'추가 피처':<20} {'mean_dist':<15} {'Δmean':<12} {'p90':<12} {'p95':<12} {'pred_std_x':<12} {'pred_std_y':<12}")
    print("-" * 80)
    
    if len(experiment_results) > 0:
        baseline_mean = experiment_results[0]['cv_mean']
        
        for result in experiment_results:
            delta_mean = result['cv_mean'] - baseline_mean
            delta_str = f"{delta_mean:+.6f}" if result['name'] != 'baseline' else "-"
            
            print(f"{result['name']:<20} {result['features']:<20} {result['cv_mean']:<15.6f} {delta_str:<12} {result['p90']:<12.6f} {result['p95']:<12.6f} {result['pred_std_x']:<12.6f} {result['pred_std_y']:<12.6f}")
        
        print("=" * 80)
        print("💡 참고:")
        print("   - Δmean: baseline 대비 변화량 (음수 = 개선, 양수 = 악화)")
        print("   - Δmean_dist가 -0.03 이하이면 의미 있는 개선으로 봅니다.")
        print("=" * 80)
        
        # 최고 성능 피처 찾기
        best_exp = min(experiment_results, key=lambda x: x['cv_mean'])
        print(f"\n🏆 최고 성능: {best_exp['name']} (mean_dist = {best_exp['cv_mean']:.6f})")
        if best_exp['name'] != 'baseline':
            improvement = baseline_mean - best_exp['cv_mean']
            print(f"   Baseline 대비 개선: {improvement:.6f} ({improvement/baseline_mean*100:.2f}%)")
        
        # 최종 결과 저장
        with open(EXPERIMENT_RESULTS_FILE, 'w', encoding='utf-8') as f:
            json.dump({
                'results': experiment_results,
                'last_updated': datetime.now().isoformat(),
                'total_time_seconds': total_time
            }, f, indent=2, ensure_ascii=False)
        print(f"\n💾 최종 결과 저장 완료: {EXPERIMENT_RESULTS_FILE}")
    else:
        print("⚠️  완료된 실험이 없습니다.")
    
else:
    # 수동 모드: 기존 로직 사용
    active_features = []
    if USE_RECENCY_WEIGHT:
        active_features.append("Recency")
    if USE_DIRECTION_SUMMARY:
        active_features.append("Direction")
    if USE_FIELD_BOUNDARY:
        active_features.append("FieldBoundary")
    if USE_SPEED_CONSTRAINT:
        active_features.append("SpeedConstraint")
    if USE_CORNER_CONTEXT:
        active_features.append("CornerContext")
    
    if active_features:
        print(f"\n🔧 피처 고도화 활성화: {', '.join(active_features)}")
        X_train_feat, X_test_feat = add_advanced_features(X_train_feat, X_test_feat, X_train, X_test, events, K)
    else:
        print("\n📊 Baseline 모드 (추가 피처 없음)")

# ----------------------
# 11-1. (옵션) CV로 OOF 점수 확인 (수동 모드용)
# ----------------------
if USE_CV and not RUN_FEATURE_EXPERIMENTS:
    if USE_ENSEMBLE:
        # OOF 앙상블 CV
        kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
        
        ensemble_results = []
        oof_pred_x_ensemble = np.zeros(len(X_train_feat), dtype=float)
        oof_pred_y_ensemble = np.zeros(len(X_train_feat), dtype=float)
        
        model_idx = 0
        for seed in ENSEMBLE_SEEDS:
            for num_leaves in ENSEMBLE_NUM_LEAVES:
                for min_child_samples in ENSEMBLE_MIN_CHILD_SAMPLES:
                    model_idx += 1
                    LGB_PARAMS = LGB_PARAMS_BASE.copy()
                    LGB_PARAMS["random_state"] = seed
                    LGB_PARAMS["num_leaves"] = num_leaves
                    LGB_PARAMS["min_child_samples"] = min_child_samples
                    
                    oof_pred_x_local = np.zeros(len(X_train_feat), dtype=float)
                    oof_pred_y_local = np.zeros(len(X_train_feat), dtype=float)
                    fold_scores = []
                    
                    print(f"\n{'='*80}")
                    print(f"📌 앙상블 모델 {model_idx}: seed={seed}, num_leaves={num_leaves}, min_child_samples={min_child_samples}")
                    print(f"{'='*80}")
                    
                    for fold, (tr_idx, va_idx) in enumerate(kf.split(X_train_feat), 1):
                        X_tr, X_va = X_train_feat.iloc[tr_idx], X_train_feat.iloc[va_idx]
                        y_tr_x, y_va_x = y_train_x[tr_idx], y_train_x[va_idx]
                        y_tr_y, y_va_y = y_train_y[tr_idx], y_train_y[va_idx]
                        
                        model_x = LGBMRegressor(**LGB_PARAMS)
                        model_x.fit(X_tr, y_tr_x, eval_set=[(X_va, y_va_x)], eval_metric="rmse", callbacks=[])
                        
                        model_y = LGBMRegressor(**LGB_PARAMS)
                        model_y.fit(X_tr, y_tr_y, eval_set=[(X_va, y_va_y)], eval_metric="rmse", callbacks=[])
                        
                        pred_va_x = np.clip(model_x.predict(X_va), 0, 105)
                        pred_va_y = np.clip(model_y.predict(X_va), 0, 68)
                        
                        oof_pred_x_local[va_idx] = pred_va_x
                        oof_pred_y_local[va_idx] = pred_va_y
                        
                        fold_euc = euclidean_mean(y_va_x, y_va_y, pred_va_x, pred_va_y)
                        fold_scores.append(fold_euc)
                        print(f"[Fold {fold}] mean Euclidean = {fold_euc:.6f}")
                    
                    cv_mean = float(np.mean(fold_scores))
                    cv_std = float(np.std(fold_scores))
                    
                    ensemble_results.append({
                        "seed": seed,
                        "num_leaves": num_leaves,
                        "min_child_samples": min_child_samples,
                        "cv_mean": cv_mean,
                        "cv_std": cv_std
                    })
                    
                    oof_pred_x_ensemble += oof_pred_x_local
                    oof_pred_y_ensemble += oof_pred_y_local
                    
                    print(f"✅ 모델 {model_idx} CV 완료 | mean Euclidean = {cv_mean:.6f} ± {cv_std:.6f}")
        
        # 앙상블 평균
        oof_pred_x_ensemble /= len(ensemble_results)
        oof_pred_y_ensemble /= len(ensemble_results)
        
        # 앙상블 OOF 점수 계산 (상세 지표)
        ensemble_cv_mean = euclidean_mean(y_train_x, y_train_y, oof_pred_x_ensemble, oof_pred_y_ensemble)
        
        # 상세 지표 계산
        dist_ensemble = np.sqrt((oof_pred_x_ensemble - y_train_x)**2 + (oof_pred_y_ensemble - y_train_y)**2)
        p90_ensemble = np.percentile(dist_ensemble, 90)
        p95_ensemble = np.percentile(dist_ensemble, 95)
        pred_std_x_ensemble = np.std(oof_pred_x_ensemble)
        pred_std_y_ensemble = np.std(oof_pred_y_ensemble)
        
        print(f"\n{'='*80}")
        print("📊 앙상블 결과 요약")
        print(f"{'='*80}")
        print(f"{'Model':<5} {'Seed':<8} {'Num_Leaves':<12} {'Min_Child':<12} {'CV_Mean':<15} {'CV_Std':<12}")
        print("-" * 80)
        for i, r in enumerate(ensemble_results, 1):
            print(f"{i:<5} {r['seed']:<8} {r['num_leaves']:<12} {r['min_child_samples']:<12} {r['cv_mean']:<15.6f} {r['cv_std']:<12.6f}")
        print("-" * 80)
        print(f"🎯 앙상블 OOF 성능:")
        print(f"   - mean_dist: {ensemble_cv_mean:.6f}")
        print(f"   - p90: {p90_ensemble:.6f}")
        print(f"   - p95: {p95_ensemble:.6f}")
        print(f"   - pred_std_x: {pred_std_x_ensemble:.6f}, pred_std_y: {pred_std_y_ensemble:.6f}")
        
        # 활성화된 피처 정보 출력
        if active_features:
            print(f"\n📌 활성화된 피처: {', '.join(active_features)}")
        else:
            print(f"\n📌 Baseline 모드")
        print(f"{'='*80}")
        
        # 최종 학습 (전체 train) - 앙상블
        print("\n" + "=" * 80)
        print("📌 전체 Train으로 최종 앙상블 모델 학습 후 Test 예측 시작...")
        print("=" * 80)
        
        pred_x_ensemble = np.zeros(len(X_test_feat), dtype=float)
        pred_y_ensemble = np.zeros(len(X_test_feat), dtype=float)
        
        for seed in ENSEMBLE_SEEDS:
            for num_leaves in ENSEMBLE_NUM_LEAVES:
                for min_child_samples in ENSEMBLE_MIN_CHILD_SAMPLES:
                    LGB_PARAMS = LGB_PARAMS_BASE.copy()
                    LGB_PARAMS["random_state"] = seed
                    LGB_PARAMS["num_leaves"] = num_leaves
                    LGB_PARAMS["min_child_samples"] = min_child_samples
                    
                    final_model_x = LGBMRegressor(**LGB_PARAMS)
                    final_model_x.fit(X_train_feat, y_train_x)
                    
                    final_model_y = LGBMRegressor(**LGB_PARAMS)
                    final_model_y.fit(X_train_feat, y_train_y)
                    
                    pred_x_ensemble += np.clip(final_model_x.predict(X_test_feat), 0, 105)
                    pred_y_ensemble += np.clip(final_model_y.predict(X_test_feat), 0, 68)
        
        pred_x = pred_x_ensemble / len(ensemble_results)
        pred_y = pred_y_ensemble / len(ensemble_results)
        
    else:
        # 단일 모델 CV
        kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
        
        oof_pred_x = np.zeros(len(X_train_feat), dtype=float)
        oof_pred_y = np.zeros(len(X_train_feat), dtype=float)
        fold_scores = []
        
        LGB_PARAMS = LGB_PARAMS_BASE.copy()
        LGB_PARAMS["random_state"] = SEED
        
        print("=" * 80)
        print(f"📌 LGBM CV 시작: n_splits={N_SPLITS}, K={K}")
        print("=" * 80)
        
        for fold, (tr_idx, va_idx) in enumerate(kf.split(X_train_feat), 1):
            X_tr, X_va = X_train_feat.iloc[tr_idx], X_train_feat.iloc[va_idx]
            y_tr_x, y_va_x = y_train_x[tr_idx], y_train_x[va_idx]
            y_tr_y, y_va_y = y_train_y[tr_idx], y_train_y[va_idx]
            
            model_x = LGBMRegressor(**LGB_PARAMS)
            model_x.fit(X_tr, y_tr_x, eval_set=[(X_va, y_va_x)], eval_metric="rmse", callbacks=[])
            
            model_y = LGBMRegressor(**LGB_PARAMS)
            model_y.fit(X_tr, y_tr_y, eval_set=[(X_va, y_va_y)], eval_metric="rmse", callbacks=[])
            
            pred_va_x = np.clip(model_x.predict(X_va), 0, 105)
            pred_va_y = np.clip(model_y.predict(X_va), 0, 68)
            
            oof_pred_x[va_idx] = pred_va_x
            oof_pred_y[va_idx] = pred_va_y
            
            fold_euc = euclidean_mean(y_va_x, y_va_y, pred_va_x, pred_va_y)
            fold_scores.append(fold_euc)
            print(f"[Fold {fold}] mean Euclidean = {fold_euc:.6f}")
        
        cv_mean = float(np.mean(fold_scores))
        cv_std = float(np.std(fold_scores))
        
        print("=" * 80)
        print(f"✅ CV 완료 | mean Euclidean = {cv_mean:.6f} ± {cv_std:.6f}")
        print("=" * 80)
        
        # 최종 학습 (전체 train) - 단일 모델
        print("\n" + "=" * 80)
        print("📌 전체 Train으로 최종 모델 학습 후 Test 예측 시작...")
        print("=" * 80)
        
        final_model_x = LGBMRegressor(**LGB_PARAMS)
        final_model_x.fit(X_train_feat, y_train_x)
        
        final_model_y = LGBMRegressor(**LGB_PARAMS)
        final_model_y.fit(X_train_feat, y_train_y)
        
        pred_x = np.clip(final_model_x.predict(X_test_feat), 0, 105)
        pred_y = np.clip(final_model_y.predict(X_test_feat), 0, 68)
    
    # 자동 실험 모드에서는 제출 파일 생성 안 함 (비교만)
    if RUN_FEATURE_EXPERIMENTS:
        print("\n" + "=" * 80)
        print("✅ 자동 실험 완료 - 제출 파일은 생성하지 않습니다")
        print("=" * 80)
        print("💡 최고 성능 피처를 확인한 후, 수동으로 플래그를 설정하고")
        print("   RUN_FEATURE_EXPERIMENTS = False로 변경하여 제출 파일을 생성하세요.")
        print("=" * 80)
    else:
        # 수동 모드: 제출 파일 생성
        # ----------------------
        # 12. submission 생성
        # ----------------------
        # 제출 파일명 생성 (피처 정보 포함)
        feature_suffix = ""
        if active_features:
            feature_suffix = "_" + "_".join(active_features).lower()
        
        if USE_ENSEMBLE:
            submission_filename = f"submission_lgbm_ensemble_k{K}{feature_suffix}.csv"
        else:
            submission_filename = f"submission_lgbm_direct_k{K}{feature_suffix}.csv"
        
        sub = sample_sub.copy()
        pred_df = X_test[["game_episode"]].copy()
        pred_df["end_x"] = pred_x
        pred_df["end_y"] = pred_y
        
        sub = sub.drop(columns=["end_x", "end_y"], errors="ignore")
        sub = sub.merge(pred_df, on="game_episode", how="left")
        sub.to_csv(submission_filename, index=False)
        
        print(f"✅ Saved: {submission_filename}")
        print("🎉 완료!")

# ----------------------
# 13. 실험 결과 요약 출력 (피처 고도화 실험용)
# ----------------------
if USE_CV and USE_ENSEMBLE:
    print(f"\n{'='*80}")
    print("📊 실험 결과 요약 (피처 고도화 실험)")
    print(f"{'='*80}")
    print(f"{'실험명':<20} {'추가 피처':<20} {'mean_dist':<15} {'p90':<12} {'p95':<12} {'pred_std_x':<12} {'pred_std_y':<12}")
    print("-" * 80)
    
    exp_name = "baseline" if not active_features else "_".join(active_features).lower()
    features_str = "없음" if not active_features else ", ".join(active_features)
    
    print(f"{exp_name:<20} {features_str:<20} {ensemble_cv_mean:<15.6f} {p90_ensemble:<12.6f} {p95_ensemble:<12.6f} {pred_std_x_ensemble:<12.6f} {pred_std_y_ensemble:<12.6f}")
    print(f"{'='*80}")
    print("💡 참고: Δmean_dist가 0.03 이상이면 의미 있는 개선으로 봅니다.")
    print(f"{'='*80}")


✓ 핵심 피처 3개 추가 완료 (dist_to_sideline, angle_to_goal_center, time_pos_inter)


/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_90174/3983062393.py:276: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lastK = lastK.groupby("game_episode", group_keys=False).apply(assign_pos_in_K)



🚀 자동화된 피처 실험 파이프라인 시작
총 6개 실험 실행 예정

📌 실험 1/6: baseline
📊 Baseline 모드 (추가 피처 없음)
  📦 피처 생성 중...
  ✓ 피처 생성 완료 (0.0초) | Train: 808 features, Test: 808 features
  🔄 CV 학습 시작 (n_splits=5)...
    └─ Fold 1/5 학습 중... 

/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_90174/3983062393.py:347: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train_feat = X_train_feat.fillna(0)
/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_90174/3983062393.py:348: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_test_feat = X_test_feat.fillna(0)


mean_dist = 14.662034 (285.6초)
    └─ Fold 2/5 학습 중... mean_dist = 14.732737 (277.3초)
    └─ Fold 3/5 학습 중... mean_dist = 14.567390 (322.2초)
    └─ Fold 4/5 학습 중... mean_dist = 14.277168 (409.1초)
    └─ Fold 5/5 학습 중... mean_dist = 14.650447 (345.1초)
  ✓ 실험 완료 (총 1639.4초)
  💾 결과 저장 완료: feature_experiment_results.json
✅ baseline 완료 | mean_dist = 14.577955 ± 0.159290
   p90 = 29.223168, p95 = 36.793945
  ⏱️  예상 남은 시간: 약 136.6분 (5개 실험 남음)

📌 실험 2/6: Recency
🔧 활성화된 피처: Recency
  📦 피처 생성 중...
  ✓ Recency 가중 피처 추가 중...
  ✓ 피처 생성 완료 (0.2초) | Train: 924 features, Test: 924 features
  🔄 CV 학습 시작 (n_splits=5)...
    └─ Fold 1/5 학습 중... mean_dist = 14.771770 (444.8초)
    └─ Fold 2/5 학습 중... mean_dist = 14.735575 (342.2초)
    └─ Fold 3/5 학습 중... mean_dist = 14.616510 (338.9초)
    └─ Fold 4/5 학습 중... mean_dist = 14.249585 (353.3초)
    └─ Fold 5/5 학습 중... mean_dist = 14.693172 (350.9초)
  ✓ 실험 완료 (총 1830.4초)
  💾 결과 저장 완료: feature_experiment_results.json
✅ Recency 완료 | mean_dist = 14.613322 ± 0.189073